# Call peaks with ```scPrinter```

- Function to use: [scprinter.pp.call_peaks](https://ruochiz.com/scprinter_doc/reference/_autosummary/scprinter.pp.call_peaks.html)
- Tutorial to follow: [scPrinter PBMC scATAC-seq tutorial](https://ruochiz.com/scprinter_doc/tutorials/PBMC_scATAC_tutorial.html#Now-let's-use-scPrinter-for-some-basic-exploratory-analysis-to-get-a-better-idea-of-the-dataset)

## 0. Imports

In [1]:
%load_ext autoreload
%autoreload 2
import scprinter as scp
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import time
import pandas as pd
import numpy as np
import os
import pickle
import torch
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42
from scanpy.plotting.palettes import zeileis_28
from tqdm.contrib.concurrent import *
from tqdm.auto import *
import anndata
import scanpy as sc
import statistics as stat
import json
import csv
import re
import copy
from sklearn.preprocessing import OneHotEncoder

In [2]:
import snapatac2 as snap

In [3]:
scp.__version__

'1.0.0a'

### 0.1 Setup

In [4]:
# Specify the reference genome. This must match that of your ATAC fragments file
genome = scp.genome.mm10

genome

Define objective

In [5]:
chromVAR_or_seq2PRINT = 'chromvar'

## 1. Paths

### 1.1 Data directories

In [6]:
master_data_dir = '/bap/bap/collab_asthma_multiome/'

In [7]:
# outputs
printer_h5ad_output_dir = os.path.join(master_data_dir, 'ATAC', '2_Analysis_Outputs', '1b_ChromVAR_scPrinter_object')
scprinter_obj_path = os.path.join(printer_h5ad_output_dir, 'Asthma_Multiome_Collab_scPrinter.h5ad')

output_dir = os.path.join(master_data_dir, 'ATAC', '2_Analysis_Outputs', f'1c_{chromVAR_or_seq2PRINT}_scPrinter_MACS_peaks')

# if the output directory does not exist, create it
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [8]:
output_dir

'/bap/bap/collab_asthma_multiome/ATAC/2_Analysis_Outputs/1c_chromvar_scPrinter_MACS_peaks'

### 1.2 Prep paths

In [9]:
# Create small lambda function to get the path to the data, input variable being sample name
get_condition_fragments_path = lambda sample_name_bc, sample_name_frag: os.path.join(master_data_dir, 'ATAC', 'ATACFragmentFiles_Asthma', sample_name_bc, f'{sample_name_frag}_atac_fragments.tsv.gz')
get_condition_valid_barcodes_path = lambda sample_name: os.path.join(master_data_dir, 'outputs', 'ATAC', '2_Analysis_Outputs', '1a_ChromVAR_Inputs', f'{sample_name}_valid_barcodes.txt')

In [10]:
# Sample names
sample_names_bc = ['NT',
                'OVA_C',
                'OVA',
                'PBS_C',
                'PBS'
                ]

# on-disk fragments files are named slightly differently
sample_names_load_fragments = ['NT',
                                'OVAC',
                                'OVA',
                                'PBSC',
                                'PBS'
                                ]

In [11]:
# to per-condition fragments

fragment_paths_l = []
valid_barcodes_l = []   # order-matched to fragment_paths_l
for sample_name_fragments_i, sample_name_bc_i in zip(sample_names_load_fragments, sample_names_bc):
    fragment_paths_l.append(get_condition_fragments_path(sample_name_bc_i, sample_name_fragments_i))
    valid_barcodes_l.append(get_condition_valid_barcodes_path(sample_name_bc_i))

In [12]:
# TODO: you'll likely need txt files of barcodes:subtype pairings per condition too,
# when you do the manual t-test later and need to group barcodes by subtype

## 2. ```scPrinter``` analysis

### 2.1 Load the scPrinter object

When you finish using the object, **run ```printer.close()``` otherwise you won't be able to load it properly next time.**

In [13]:
printer = scp.load_printer(scprinter_obj_path, genome)

In [14]:
printer

head project
AnnData object with n_obs x n_vars = 7418 x 0 backed at '/bap/bap/collab_asthma_multiome/ATAC/2_Analysis_Outputs/1b_ChromVAR_scPrinter_object/Asthma_Multiome_Collab_scPrinter.h5ad'
    obs: 'sample', 'n_fragment', 'frac_dup', 'frac_mito', 'frag_path', 'frag_sample_name', 'tsse'
    uns: 'unique_string', 'genome', 'gff_db', 'insertion', 'footprints', 'bias_bw', 'bias_path', 'binding score', 'reference_sequences'
    obsm: 'insertion_chr18', 'insertion_chr7', 'insertion_chr8', 'insertion_chrY', 'insertion_chr4', 'insertion_chr19', 'insertion_chr12', 'insertion_chr13', 'insertion_chr5', 'insertion_chr2', 'insertion_chr3', 'insertion_chr14', 'insertion_chr6', 'insertion_chr9', 'insertion_chrX', 'insertion_chr17', 'insertion_chr1', 'insertion_chr10', 'insertion_chr11', 'insertion_chr15', 'insertion_chr16'


### 2.2 Peak calling (seq2PRINT preset)

In [15]:
# Check uniqueness of barcodes

# Load each text file of valid barcodes into a big list and then check for duplicates
valid_barcodes = []
for valid_barcodes_path_i in valid_barcodes_l:
    with open(valid_barcodes_path_i, 'r') as f:
        valid_barcodes_i = f.readlines()
        valid_barcodes_i = [x.strip() for x in valid_barcodes_i]
        valid_barcodes.extend(valid_barcodes_i)

# Check for duplicates
duplicate_barcodes = set([x for x in valid_barcodes if valid_barcodes.count(x) > 1])
if len(duplicate_barcodes) > 0:
    print(f'Found {len(duplicate_barcodes)} duplicate barcodes:')
    print(duplicate_barcodes)
else:
    print('No duplicate barcodes found')

Found 14 duplicate barcodes:
{'AAGTAGCCATAATCCG-1', 'CCTGTTGGTTAATGAC-1', 'GCGAAGTAGGCAGGTG-1', 'CAAAGGATCATGTGGT-1', 'AGTAAGTAGTTAGAGG-1', 'TTGGGCCAGGACCTGC-1', 'ACTTGTCGTTCCGGGA-1', 'CTCTAGCTCATGCTTT-1', 'GGTAACCGTGCTGTAA-1', 'CTTACCGGTTGCAATG-1', 'AATAGCTGTCAAGTAT-1', 'TCAAGGAAGCGGTTAT-1', 'GCGCGATTCCCATAGG-1', 'AGCAACAAGCCGCAGT-1'}


In [16]:
sample_names_bc

['NT', 'OVA_C', 'OVA', 'PBS_C', 'PBS']

In [17]:
valid_barcodes_l

['/bap/bap/collab_asthma_multiome/outputs/ATAC/2_Analysis_Outputs/1a_ChromVAR_Inputs/NT_valid_barcodes.txt',
 '/bap/bap/collab_asthma_multiome/outputs/ATAC/2_Analysis_Outputs/1a_ChromVAR_Inputs/OVA_C_valid_barcodes.txt',
 '/bap/bap/collab_asthma_multiome/outputs/ATAC/2_Analysis_Outputs/1a_ChromVAR_Inputs/OVA_valid_barcodes.txt',
 '/bap/bap/collab_asthma_multiome/outputs/ATAC/2_Analysis_Outputs/1a_ChromVAR_Inputs/PBS_C_valid_barcodes.txt',
 '/bap/bap/collab_asthma_multiome/outputs/ATAC/2_Analysis_Outputs/1a_ChromVAR_Inputs/PBS_valid_barcodes.txt']

In [18]:
# Read in valid_barcodes_l as a nested list

valid_bc = list(printer.obs_names)

In [19]:
len(valid_bc)

7418

We want to do ChromVAR analysis here, so let's comment out what we would do if we were preparing peaks for seq2PRINT (note: we wouldn't lump all cells together for that anyway).

In [20]:
# # Here we call peaks on all pass-QC cells across all subtypes and all conditions/samples, for ChromVAR analysis

# scp.pp.call_peaks(printer=printer,
#                   frag_file=fragment_paths_l,
#                   cell_grouping=[valid_bc], 
#                   group_names=['all'],
#                   sample_names=sample_names_bc,
#                   preset='seq2PRINT',
#                   overwrite=True,
#                   merge_across_groups=True,
#                   n_jobs=1
#                   )

In [21]:
# printer.uns["peak_calling"]

In [22]:
# # Fetched the cleaned peaks, save, it will be used in the next step
# cleaned_peaks_seq2PRINT = pd.DataFrame(printer.uns["peak_calling"]['merged'][:])
# cleaned_peaks_seq2PRINT.to_csv(f'{output_dir}/seq2PRINT_preset_Asthma_Multiome_scPrinter_cleaned_narrowPeak.bed',
#                      sep='\t', header=False, index=False)

### 2.3 Peak calling (ChromVAR preset)

In [23]:
# Call peaks using chromvar preset, this set of peak are recommended to be use as cell x peak for scATAC-seq data, or analysis
scp.pp.call_peaks(printer=printer,
                  frag_file=fragment_paths_l,
                  cell_grouping=[valid_bc], 
                  group_names=['chromvar_all'],
                  sample_names=sample_names_bc,
                  preset='chromvar',
                  overwrite=True,
                  merge_across_groups=True,
                  n_jobs=1
                  )

running macs2 with macs2 callpeak --nomodel -t /bap/bap/collab_asthma_multiome/ATAC/2_Analysis_Outputs/1b_ChromVAR_scPrinter_object/Asthma_Multiome_Collab_scPrinter_supp/chromvar_all_filtered_frag.tsv.gz --outdir /bap/bap/collab_asthma_multiome/ATAC/2_Analysis_Outputs/1b_ChromVAR_scPrinter_object/Asthma_Multiome_Collab_scPrinter_supp/macs2 -n chromvar_all -f BEDPE --nolambda --keep-dup all --call-summits --nomodel -B --SPMR --shift 75 --extsize 150 -q 0.01


INFO  @ Thu, 27 Feb 2025 14:21:05: 
# Command line: callpeak --nomodel -t /bap/bap/collab_asthma_multiome/ATAC/2_Analysis_Outputs/1b_ChromVAR_scPrinter_object/Asthma_Multiome_Collab_scPrinter_supp/chromvar_all_filtered_frag.tsv.gz --outdir /bap/bap/collab_asthma_multiome/ATAC/2_Analysis_Outputs/1b_ChromVAR_scPrinter_object/Asthma_Multiome_Collab_scPrinter_supp/macs2 -n chromvar_all -f BEDPE --nolambda --keep-dup all --call-summits --nomodel -B --SPMR --shift 75 --extsize 150 -q 0.01
# ARGUMENTS LIST:
# name = chromvar_all
# format = BEDPE
# ChIP-seq file = ['/bap/bap/collab_asthma_multiome/ATAC/2_Analysis_Outputs/1b_ChromVAR_scPrinter_object/Asthma_Multiome_Collab_scPrinter_supp/chromvar_all_filtered_frag.tsv.gz']
# control file = None
# effective genome size = 2.70e+09
# band width = 300
# model fold = [5, 50]
# qvalue cutoff = 1.00e-02
# The maximum gap between significant sites is assigned as the read length/tag size.
# The minimum length of peaks is assigned as the predicted fragme

Reading in peak summit file(s):
NOTE: Assuming all start coordinates are 0-based ..

Padding peak summits by: 400 bp on either side for
Removing peaks overlapping with blacklisted regions and out of bound peaks based on chromosome sizes ..

Filtering overlapping peaks based on peak summit score ..
round: 1 576819 peaks unresolved 324636 peaks selected
round: 2 83434 peaks unresolved 37626 peaks selected
round: 3 16765 peaks unresolved 6631 peaks selected
round: 4 3974 peaks unresolved 1475 peaks selected
round: 5 998 peaks unresolved 355 peaks selected
round: 6 260 peaks unresolved 98 peaks selected
round: 7 54 peaks unresolved 22 peaks selected
round: 8 10 peaks unresolved 3 peaks selected
round: 9 4 peaks unresolved 1 peaks selected
round: 10 1 peaks unresolved 1 peaks selected
finish clearing
finish sorting
finished summary


In [25]:
printer.uns["peak_calling"].keys()

/tmp/ipykernel_1627622/1189696960.py:1: DeprecationWarning: `Series._import_from_c` is deprecated. use _import_arrow_from_c; if you are using an extension, please compile it with latest 'pyo3-polars'
  printer.uns["peak_calling"].keys()


dict_keys(['chromvar_all', 'merged'])

In [26]:
pd.DataFrame(printer.uns["peak_calling"]['merged'][:]).shape

/tmp/ipykernel_1627622/52693986.py:1: DeprecationWarning: `Series._import_from_c` is deprecated. use _import_arrow_from_c; if you are using an extension, please compile it with latest 'pyo3-polars'
  pd.DataFrame(printer.uns["peak_calling"]['merged'][:]).shape


(370848, 8)

In [28]:
pd.DataFrame(printer.uns["peak_calling"]['chromvar_all'][:]).shape

/tmp/ipykernel_1627622/1842597009.py:1: DeprecationWarning: `Series._import_from_c` is deprecated. use _import_arrow_from_c; if you are using an extension, please compile it with latest 'pyo3-polars'
  pd.DataFrame(printer.uns["peak_calling"]['chromvar_all'][:]).shape


(583083, 10)

In [29]:
# Fetched the cleaned peaks, save, it will be used in the next step
cleaned_peaks_chromVAR = pd.DataFrame(printer.uns["peak_calling"]['merged'][:])
cleaned_peaks_chromVAR.to_csv(f'{output_dir}/chromVAR_preset_Asthma_Multiome_scPrinter_cleaned_merged_narrowPeak.bed',
                     sep='\t', header=False, index=False)

/tmp/ipykernel_1627622/2289798895.py:2: DeprecationWarning: `Series._import_from_c` is deprecated. use _import_arrow_from_c; if you are using an extension, please compile it with latest 'pyo3-polars'
  cleaned_peaks_chromVAR = pd.DataFrame(printer.uns["peak_calling"]['merged'][:])


## 3. Close ```printer``` object

In [30]:
printer.close()

# END